# Set Up

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import os
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_col

In [3]:
file_path = os.path.abspath('tablea2.csv')
file_path

'/home/jovyan/bac/tablea2.csv'

# Downloading and Assigning Regions to Table A2

In [3]:
# This is the HCD data from 2018-2023, downloaded Jan 2025

hcd_data = pd.read_csv(file_path)
hcd_data.head()

/tmp/ipykernel_89/2700009885.py:3: DtypeWarning: Columns (3,16,25,33,36,37,42,43,49,50) have mixed types. Specify dtype option on import or set low_memory=False.
  hcd_data = pd.read_csv(file_path)


,JURIS_NAME,CNTY_NAME,YEAR,PRIOR_APN,APN,STREET_ADDRESS,PROJECT_NAME,JURS_TRACKING_ID,UNIT_CAT,TENURE,...,NO_FA_DR,TERM_AFF_DR,DEM_DES_UNITS,DEM_OR_DES_UNITS,DEM_DES_UNITS_OWN_RENT,DENSITY_BONUS_TOTAL,DENSITY_BONUS_NUMBER_OTHER_INCENTIVES,DENSITY_BONUS_INCENTIVES,DENSITY_BONUS_RECEIVE_REDUCTION,NOTES
0,ADELANTO,San Bernardino,2018,NaN,312846105,15824 BELLFLOWER ST,JOSE & ODULIA LOPEZ,NaN,SFD,Owner,...,0,NaN,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
1,ADELANTO,San Bernardino,2018,NaN,312878124,11939 GAINES CT,"FRONTIER HOMES II, LLC",NaN,SFD,Owner,...,0,NaN,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
2,ADELANTO,San Bernardino,2018,NaN,312878125,11931 GAINES CT,"FRONTIER HOMES II, LLC",NaN,SFD,Owner,...,0,NaN,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
3,ADELANTO,San Bernardino,2018,NaN,312878126,11925 GAINES CT,"FRONTIER HOMES II, LLC",NaN,SFD,Owner,...,0,NaN,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
4,ADELANTO,San Bernardino,2018,NaN,312878127,11913 GAINES CT,"FRONTIER HOMES II, LLC",NaN,SFD,Owner,...,0,NaN,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN


In [4]:
# Define regions in California
regions = {
    "Bay Area": ["Alameda", "Contra Costa", "Marin", "Napa", "San Francisco", "San Mateo", "Santa Clara", "Solano", "Sonoma", "Santa Cruz"],
    "Central Region": ["Fresno", "Kern", "Kings", "Madera", "Mariposa", "Merced", "Monterey", "San Benito", "San Luis Obispo", "Stanislaus", "Tulare", "Tuolumne"],
    "Inland Deserts Region": ["Imperial", "Inyo", "Mono", "Riverside", "San Bernardino"],
    "North Central Region": ["Sacramento", "San Joaquin", "El Dorado", "Lake", "Nevada", "Placer", "Sierra", "Sutter", "Plumas", "Yolo", "Yuba",
                             "Alpine", "Amador", "Calaveras", "Butte", "Colusa", "Glenn"],
    "Northern Region": ["Del Norte", "Humboldt", "Lassen", "Mendocino", "Modoc", "Shasta", "Siskiyou", "Tehama", "Trinity"],
    "South Coast Region": ["Los Angeles", "Orange", "San Diego", "Santa Barbara", "Ventura"]
}

# Map counties to region
def get_region(county):
    for region, counties in regions.items():
        if county in counties:
            return region
    return "Other California"  # Default category for any unmatched counties

# Apply function to create Region column
hcd_data['Region'] = hcd_data['CNTY_NAME'].apply(get_region)

In [5]:
# Map UNIT_CAT to descriptive names to create Type column
unit_cat_mapping = {
    "SFD": "Single-Family Detached Unit",
    "SFA": "Single-Family Attached Unit",
    "MH": "Mobile Home Unit",
    "2 to 4": "2-, 3-, and 4-Plex Units per Structure",
    "ADU": "Accessory Dwelling Unit",
    "5+": "5 or More Units per Structure"
}

hcd_data['Type'] = hcd_data['UNIT_CAT'].map(unit_cat_mapping)

hcd_data

,JURIS_NAME,CNTY_NAME,YEAR,PRIOR_APN,APN,STREET_ADDRESS,PROJECT_NAME,JURS_TRACKING_ID,UNIT_CAT,TENURE,...,DEM_DES_UNITS,DEM_OR_DES_UNITS,DEM_DES_UNITS_OWN_RENT,DENSITY_BONUS_TOTAL,DENSITY_BONUS_NUMBER_OTHER_INCENTIVES,DENSITY_BONUS_INCENTIVES,DENSITY_BONUS_RECEIVE_REDUCTION,NOTES,Region,Type
0,ADELANTO,San Bernardino,2018,NaN,312846105,15824 BELLFLOWER ST,JOSE & ODULIA LOPEZ,NaN,SFD,Owner,...,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,Inland Deserts Region,Single-Family Detached Unit
1,ADELANTO,San Bernardino,2018,NaN,312878124,11939 GAINES CT,"FRONTIER HOMES II, LLC",NaN,SFD,Owner,...,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,Inland Deserts Region,Single-Family Detached Unit
2,ADELANTO,San Bernardino,2018,NaN,312878125,11931 GAINES CT,"FRONTIER HOMES II, LLC",NaN,SFD,Owner,...,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,Inland Deserts Region,Single-Family Detached Unit
3,ADELANTO,San Bernardino,2018,NaN,312878126,11925 GAINES CT,"FRONTIER HOMES II, LLC",NaN,SFD,Owner,...,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,Inland Deserts Region,Single-Family Detached Unit
4,ADELANTO,San Bernardino,2018,NaN,312878127,11913 GAINES CT,"FRONTIER HOMES II, LLC",NaN,SFD,Owner,...,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,Inland Deserts Region,Single-Family Detached Unit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
646935,YUCCA VALLEY,San Bernardino,2023,NaN,0598-601-10,58855 MEREDITH CT,NaN,2021-2239,SFD,Owner,...,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,Inland Deserts Region,Single-Family Detached Unit
646936,YUCCA VALLEY,San Bernardino,2023,NaN,0598-601-12,58871 MEREDITH CT,NaN,2021-2266,SFD,Owner,...,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,Inland Deserts Region,Single-Family Detached Unit
646937,YUCCA VALLEY,San Bernardino,2023,NaN,0601-021-43,57750 PAXTON RD,NaN,2023-333,SFD,Owner,...,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,Inland Deserts Region,Single-Family Detached Unit
646938,YUCCA VALLEY,San Bernardino,2023,NaN,0601-081-66,59045 WILCOX LN,NaN,2022-905,SFD,Owner,...,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,Inland Deserts Region,Single-Family Detached Unit


# Tableau 2013-2023 Chart

In [6]:
# Ensure NO_BUILDING_PERMITS is numeric
hcd_data['NO_BUILDING_PERMITS'] = pd.to_numeric(hcd_data['NO_BUILDING_PERMITS'], errors='coerce')

# Group by Type, Region, and Year to calculate the total number of units built
share_of_housing_types = hcd_data.groupby(['Type', 'Region', 'YEAR'])['NO_BUILDING_PERMITS'].sum().reset_index()

# Calculate total units built per year across all regions (California total)
california_totals = share_of_housing_types.groupby(['Type', 'YEAR'])['NO_BUILDING_PERMITS'].sum().reset_index()

# Assign 'California' as the region for these totals
california_totals['Region'] = 'California'

# Filter only Bay Area data
bay_area_data = share_of_housing_types[share_of_housing_types['Region'] == 'Bay Area']

# Combine Bay Area and California totals into a single long-format DataFrame
long_format_data = pd.concat([bay_area_data, california_totals], ignore_index=True)

long_format_data

,Type,Region,YEAR,NO_BUILDING_PERMITS
0,"2-, 3-, and 4-Plex Units per Structure",Bay Area,2018,691
1,"2-, 3-, and 4-Plex Units per Structure",Bay Area,2019,800
2,"2-, 3-, and 4-Plex Units per Structure",Bay Area,2020,432
3,"2-, 3-, and 4-Plex Units per Structure",Bay Area,2021,554
4,"2-, 3-, and 4-Plex Units per Structure",Bay Area,2022,254
...,...,...,...,...
67,Single-Family Detached Unit,California,2019,48633
68,Single-Family Detached Unit,California,2020,45579
69,Single-Family Detached Unit,California,2021,55065
70,Single-Family Detached Unit,California,2022,47167


In [7]:
# Added 2013-2017 from Abby's Data to the table above and reuploaded it

long_format_data_2013_2023 = pd.read_csv('/home/jovyan/bac/2013-2023_Chart.csv')
long_format_data_2013_2023.head()

,Type,Region,YEAR,NO_BUILDING_PERMITS
0,"2-, 3-, and 4-Plex Units per Structure",Bay Area,2018,691.0
1,"2-, 3-, and 4-Plex Units per Structure",Bay Area,2019,800.0
2,"2-, 3-, and 4-Plex Units per Structure",Bay Area,2020,432.0
3,"2-, 3-, and 4-Plex Units per Structure",Bay Area,2021,554.0
4,"2-, 3-, and 4-Plex Units per Structure",Bay Area,2022,254.0


In [8]:
# Calculate total permits per Region and Type
total_by_region_type = long_format_data_2013_2023.groupby(['Region','YEAR'])['NO_BUILDING_PERMITS'].transform('sum')

# Compute share
long_format_data_2013_2023['Share'] = (long_format_data_2013_2023['NO_BUILDING_PERMITS'] / total_by_region_type * 100).round(1)
long_format_data_2013_2023
#long_format_data_2013_2023.to_csv("2013-2023chartfortableau.csv", index=False)

,Type,Region,YEAR,NO_BUILDING_PERMITS,Share
0,"2-, 3-, and 4-Plex Units per Structure",Bay Area,2018,691.0,2.3
1,"2-, 3-, and 4-Plex Units per Structure",Bay Area,2019,800.0,2.8
2,"2-, 3-, and 4-Plex Units per Structure",Bay Area,2020,432.0,1.8
3,"2-, 3-, and 4-Plex Units per Structure",Bay Area,2021,554.0,2.1
4,"2-, 3-, and 4-Plex Units per Structure",Bay Area,2022,254.0,0.8
...,...,...,...,...,...
127,Mobile Home Unit,California,2013,903.0,1.1
128,Single-Family Attached Unit,Bay Area,2013,NaN,NaN
129,Single-Family Attached Unit,California,2013,NaN,NaN
130,Single-Family Detached Unit,Bay Area,2013,3929.0,24.6


In [9]:
# Load data
time_chart_updated = pd.read_csv('/home/jovyan/bac/2013-2023_Chart_updated.csv')

# Select the correct household count based on region
time_chart_updated['Households'] = time_chart_updated.apply(
    lambda row: row['bayarea_households'] if row['Region'] == 'Bay Area' else row['ca_households'], axis=1
)

# Calculate total permits per Region and Year
total_permits_by_region = time_chart_updated.groupby(['Region', 'YEAR'])['NO_BUILDING_PERMITS'].transform('sum')

# Compute normalized share (percentage of total permits in the region)
time_chart_updated['Normalized_Share'] = (time_chart_updated['NO_BUILDING_PERMITS'] / total_permits_by_region * 100).round(2)

# Compute permits per 10,000 households (if still needed)
time_chart_updated['Share_per_10K_Households'] = (time_chart_updated['NO_BUILDING_PERMITS'] / time_chart_updated['Households'] * 10000).round(2)

# Display the corrected data
time_chart_updated

,Type,Region,YEAR,NO_BUILDING_PERMITS,ca_households,bayarea_households,Households,Normalized_Share,Share_per_10K_Households
0,"2-, 3-, and 4-Plex Units per Structure",Bay Area,2013,691.0,12542460,2613055,2613055,4.33,2.64
1,"2-, 3-, and 4-Plex Units per Structure",California,2013,4111.0,12542460,2613055,12542460,5.11,3.28
2,5 or More Units per Structure,Bay Area,2013,10975.0,12542460,2613055,2613055,68.74,42.00
3,5 or More Units per Structure,California,2013,39837.0,12542460,2613055,12542460,49.49,31.76
4,Accessory Dwelling Unit,Bay Area,2013,234.0,12542460,2613055,2613055,1.47,0.90
...,...,...,...,...,...,...,...,...,...
127,5 or More Units per Structure,California,2023,71391.0,13434847,2785482,13434847,43.73,53.14
128,Accessory Dwelling Unit,California,2023,35393.0,13434847,2785482,13434847,21.68,26.34
129,Mobile Home Unit,California,2023,2440.0,13434847,2785482,13434847,1.49,1.82
130,Single-Family Attached Unit,California,2023,5563.0,13434847,2785482,13434847,3.41,4.14


In [10]:
# Perform the merge on the key columns
barchart_all = time_chart_updated.merge(
    long_format_data_2013_2023[['Type', 'Region', 'YEAR', 'NO_BUILDING_PERMITS', 'Share']],
    on=['Type', 'Region', 'YEAR', 'NO_BUILDING_PERMITS'],
    how='left'
)

# Rename the 'Share' column to 'share_notnormalized'
barchart_all.rename(columns={'Share_y': 'share_notnormalized'}, inplace=True)
barchart_all.rename(columns={'Share_x': 'share_normalized'}, inplace=True)

# Display the resulting dataframe
barchart_all
#barchart_all.to_csv('barchart_all_2.csv')

,Type,Region,YEAR,NO_BUILDING_PERMITS,ca_households,bayarea_households,Households,Normalized_Share,Share_per_10K_Households,Share
0,"2-, 3-, and 4-Plex Units per Structure",Bay Area,2013,691.0,12542460,2613055,2613055,4.33,2.64,4.3
1,"2-, 3-, and 4-Plex Units per Structure",California,2013,4111.0,12542460,2613055,12542460,5.11,3.28,5.1
2,5 or More Units per Structure,Bay Area,2013,10975.0,12542460,2613055,2613055,68.74,42.00,68.7
3,5 or More Units per Structure,California,2013,39837.0,12542460,2613055,12542460,49.49,31.76,49.5
4,Accessory Dwelling Unit,Bay Area,2013,234.0,12542460,2613055,2613055,1.47,0.90,1.5
...,...,...,...,...,...,...,...,...,...,...
127,5 or More Units per Structure,California,2023,71391.0,13434847,2785482,13434847,43.73,53.14,43.7
128,Accessory Dwelling Unit,California,2023,35393.0,13434847,2785482,13434847,21.68,26.34,21.7
129,Mobile Home Unit,California,2023,2440.0,13434847,2785482,13434847,1.49,1.82,1.5
130,Single-Family Attached Unit,California,2023,5563.0,13434847,2785482,13434847,3.41,4.14,3.4


# 2023 Data

In [4]:
hcd_2023_geocoded = pd.read_csv('/home/jovyan/bac/hcd_2023_redlined.csv')
hcd_2023_geocoded.head()

/tmp/ipykernel_157/4022310405.py:1: DtypeWarning: Columns (7,44,46) have mixed types. Specify dtype option on import or set low_memory=False.
  hcd_2023_geocoded = pd.read_csv('/home/jovyan/bac/hcd_2023_redlined.csv')


,Join_Count,TARGET_FID,Join_Count_1,TARGET_FID_1,JURIS_NAME,CNTY_NAME,YEAR,PRIOR_APN,APN,STREET_ADDRESS,...,city_1,state_1,city_survey,category,grade,label,residential,commercial,industrial,fill
0,1,1,0,1,ADELANTO,San Bernardino,2023,NaN,310310314,10352 San Marcos Ct,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,0,2,ADELANTO,San Bernardino,2023,NaN,310310315,10368 San Marcos Ct,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,3,0,3,ADELANTO,San Bernardino,2023,NaN,310310320,18786 Hampton Ln,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,4,0,4,ADELANTO,San Bernardino,2023,NaN,310310320,18792 Hampton Ln,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,5,0,5,ADELANTO,San Bernardino,2023,NaN,310310329,10353 San Marcos Ct,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# Convert ZIP codes to string and remove .0
#hcd_2023_geocoded = hcd_2023_geocoded.dropna(subset=['Zip'])  # Remove NaNs
#hcd_2023_geocoded['Zip'] = hcd_2023_geocoded['Zip'].astype(int).astype(str)  # Convert to int then string
#hcd_2023_geocoded

In [13]:
# Add the 'Region' and 'Type' column for 2023
#hcd_2023_geocoded['Region'] = hcd_2023_geocoded['CNTY_NAME'].apply(lambda x: 'Bay Area' if x in bay_area_counties else 'California')

#hcd_2023_geocoded['Type'] = hcd_2023_geocoded['UNIT_CAT'].map(unit_cat_mapping)


In [14]:
hcd_data = hcd_data[hcd_data['YEAR'] != 2023]

In [15]:
# Append new data (2018-2023)
hcd_all = pd.concat([hcd_data, hcd_2023_geocoded], ignore_index=True)
hcd_all.head()

,JURIS_NAME,CNTY_NAME,YEAR,PRIOR_APN,APN,STREET_ADDRESS,PROJECT_NAME,JURS_TRACKING_ID,UNIT_CAT,TENURE,...,city_1,state_1,city_survey,category,grade,label,residential,commercial,industrial,fill
0,ADELANTO,San Bernardino,2018,NaN,312846105,15824 BELLFLOWER ST,JOSE & ODULIA LOPEZ,NaN,SFD,Owner,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ADELANTO,San Bernardino,2018,NaN,312878124,11939 GAINES CT,"FRONTIER HOMES II, LLC",NaN,SFD,Owner,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ADELANTO,San Bernardino,2018,NaN,312878125,11931 GAINES CT,"FRONTIER HOMES II, LLC",NaN,SFD,Owner,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ADELANTO,San Bernardino,2018,NaN,312878126,11925 GAINES CT,"FRONTIER HOMES II, LLC",NaN,SFD,Owner,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ADELANTO,San Bernardino,2018,NaN,312878127,11913 GAINES CT,"FRONTIER HOMES II, LLC",NaN,SFD,Owner,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Census Data

In [16]:
acs = pd.read_csv('/home/jovyan/bac/B03002_2023.csv')
acs.head()

,GEOID,Geography,tot_pop,tot_nothisp,tot_white_nonhisp,tot_black,tot_asian,tot_hisp,tot_poc,share_whitee,share_black,share_asian,share_hisp,share_poc,majority_poc_flag,tot_native,tot_pac_isl,tot_otherrace,tot_tworaces
0,6001400100,Census Tract 4001; Alameda County; California,3094,2894,2107,137,462,200,987,68.09954751,4.427925016,14.9321267,6.464124111,31.90045249,0,0,0,25,163
1,6001400200,Census Tract 4002; Alameda County; California,2093,1897,1408,43,256,196,685,67.27185858,2.054467272,12.23124701,9.364548495,32.72814142,0,14,0,7,169
2,6001400300,Census Tract 4003; Alameda County; California,5727,5230,3365,524,609,497,2362,58.7567662,9.149642046,10.63383971,8.678191025,41.2432338,0,21,0,47,664
3,6001400400,Census Tract 4004; Alameda County; California,4395,3791,2645,433,422,604,1750,60.18202503,9.852104664,9.60182025,13.74288965,39.81797497,0,0,0,21,270
4,6001400500,Census Tract 4005; Alameda County; California,3822,3265,1696,911,306,557,2126,44.37467295,23.83568812,8.006279435,14.57352172,55.62532705,1,4,7,0,341


In [17]:
hcd_2023_geocoded = pd.read_csv('/home/jovyan/bac/merged_hcd_2023_acs.csv')
hcd_2023_geocoded.head()

/tmp/ipykernel_89/147427316.py:1: DtypeWarning: Columns (8,45,47,105,106,107,108,109,110) have mixed types. Specify dtype option on import or set low_memory=False.
  hcd_2023_geocoded = pd.read_csv('/home/jovyan/bac/merged_hcd_2023_acs.csv')


,Unnamed: 0,Join_Count,TARGET_FID,Join_Count_1,TARGET_FID_1,JURIS_NAME,CNTY_NAME,YEAR,PRIOR_APN,APN,...,share_whitee,share_black,share_asian,share_hisp,share_poc,majority_poc_flag,tot_native,tot_pac_isl,tot_otherrace,tot_tworaces
0,0,1,1,0,1,ADELANTO,San Bernardino,2023,NaN,310310314,...,13.048934,16.348808,0.890841,66.474279,86.951066,1.0,61.0,0.0,197.0,0.0
1,1,1,2,0,2,ADELANTO,San Bernardino,2023,NaN,310310315,...,13.048934,16.348808,0.890841,66.474279,86.951066,1.0,61.0,0.0,197.0,0.0
2,2,1,3,0,3,ADELANTO,San Bernardino,2023,NaN,310310320,...,13.048934,16.348808,0.890841,66.474279,86.951066,1.0,61.0,0.0,197.0,0.0
3,3,1,4,0,4,ADELANTO,San Bernardino,2023,NaN,310310320,...,13.048934,16.348808,0.890841,66.474279,86.951066,1.0,61.0,0.0,197.0,0.0
4,4,1,5,0,5,ADELANTO,San Bernardino,2023,NaN,310310329,...,13.048934,16.348808,0.890841,66.474279,86.951066,1.0,61.0,0.0,197.0,0.0


In [18]:
hcd_2023_shares = hcd_2023_geocoded.drop(hcd_2023_geocoded.columns[:5], axis=1)
#hcd_2023_shares.to_csv('hcd_2023_shares.csv')

# Adding Income and Poverty ACS Data

In [19]:
acs_income = pd.read_csv('/home/jovyan/bac/income_acs.csv')
acs_income.head()

,GEO_ID,NAME,tot_households,less_than_10k,inc_10k_to_15k,inc_15k_to_25k,inc_25k_to_35k,inc_35k_to_50k,inc_50k_to_75k,inc_75k_to_100k,inc_100k_to_150k,inc_150k_to_200k,inc_200k_or_more,median_income,mean_income
0,6001400100,Census Tract 4001; Alameda County; California,1316,6,0,1.8,2.1,1.7,4,1.3,13.9,8.9,60.2,"250,000+",306330
1,6001400200,Census Tract 4002; Alameda County; California,861,1.4,1.7,1.6,0,0.8,5.7,6.3,11.4,15.2,55.9,225880,278639
2,6001400300,Census Tract 4003; Alameda County; California,2713,4.4,3.5,3.6,3.7,2.9,7.6,11,12.5,9.6,41.2,157731,220895
3,6001400400,Census Tract 4004; Alameda County; California,1803,0.4,0.4,7,4,1.4,9.3,4.3,17.1,16.5,39.4,159612,263959
4,6001400500,Census Tract 4005; Alameda County; California,1655,1.5,2.2,8.8,1.3,8.3,20.2,7.9,13.5,11,25.3,96250,145831


In [20]:
acs_poverty = pd.read_csv('/home/jovyan/bac/poverty_acs.csv')
acs_poverty.head()

,GEO_ID,NAME,total_pop,pop_below_pov_level,p_below_pov_level
0,6001400100,Census Tract 4001; Alameda County; California,3075,134,4.4
1,6001400200,Census Tract 4002; Alameda County; California,2093,164,7.8
2,6001400300,Census Tract 4003; Alameda County; California,5707,310,5.4
3,6001400400,Census Tract 4004; Alameda County; California,4351,343,7.9
4,6001400500,Census Tract 4005; Alameda County; California,3816,397,10.4


In [21]:
# Merge pov and income datasets
acs_pov_inc = pd.merge(acs_poverty, acs_income, on=['GEO_ID','NAME'], how='inner')
acs_pov_inc.head()

,GEO_ID,NAME,total_pop,pop_below_pov_level,p_below_pov_level,tot_households,less_than_10k,inc_10k_to_15k,inc_15k_to_25k,inc_25k_to_35k,inc_35k_to_50k,inc_50k_to_75k,inc_75k_to_100k,inc_100k_to_150k,inc_150k_to_200k,inc_200k_or_more,median_income,mean_income
0,6001400100,Census Tract 4001; Alameda County; California,3075,134,4.4,1316,6,0,1.8,2.1,1.7,4,1.3,13.9,8.9,60.2,"250,000+",306330
1,6001400200,Census Tract 4002; Alameda County; California,2093,164,7.8,861,1.4,1.7,1.6,0,0.8,5.7,6.3,11.4,15.2,55.9,225880,278639
2,6001400300,Census Tract 4003; Alameda County; California,5707,310,5.4,2713,4.4,3.5,3.6,3.7,2.9,7.6,11,12.5,9.6,41.2,157731,220895
3,6001400400,Census Tract 4004; Alameda County; California,4351,343,7.9,1803,0.4,0.4,7,4,1.4,9.3,4.3,17.1,16.5,39.4,159612,263959
4,6001400500,Census Tract 4005; Alameda County; California,3816,397,10.4,1655,1.5,2.2,8.8,1.3,8.3,20.2,7.9,13.5,11,25.3,96250,145831


# Merge Ratio Income / Poverty with the dataset

In [22]:
ratio_inc_pov = pd.read_csv('/home/jovyan/bac/ratio_inc_pov.csv')
ratio_inc_pov.head()

,GEO_ID,NAME,total_pop,ratio_inc_pov_under_one,p_ratio_inc_pov_under_one
0,6001400100,Census Tract 4001; Alameda County; California,553,0,0.00
1,6001400200,Census Tract 4002; Alameda County; California,361,0,0.00
2,6001400300,Census Tract 4003; Alameda County; California,978,0,0.00
3,6001400400,Census Tract 4004; Alameda County; California,940,110,11.70
4,6001400500,Census Tract 4005; Alameda County; California,510,90,17.65


In [23]:
pop_household_med_income = pd.read_csv('/home/jovyan/bac/pop_household_med_income.csv')
pop_household_med_income.head()

,Label (Grouping),"Alameda County, California","Alpine County, California","Amador County, California","Butte County, California","Calaveras County, California","Colusa County, California","Contra Costa County, California","Del Norte County, California","El Dorado County, California",...,"Sonoma County, California","Stanislaus County, California","Sutter County, California","Tehama County, California","Trinity County, California","Tulare County, California","Tuolumne County, California","Ventura County, California","Yolo County, California","Yuba County, California"
0,Total Population,"1,651,949","1,695","41,029","209,470","45,995","21,895","1,161,458","27,293","192,299",...,"485,642","552,250","98,971","65,520","15,886","475,774","54,873","838,259","217,782","83,079"
1,Total Households,"593,117",473,"16,066","82,345","17,897","7,466","411,662","9,621","75,719",...,"190,498","176,457","33,240","24,526","5,373","142,026","22,809","278,045","76,640","28,063"
2,Median income (dollars),"126,240","110,781","81,526","68,574","79,877","75,149","125,727","66,780","106,190",...,"102,840","79,661","75,450","61,834","53,498","69,489","72,259","107,327","88,818","73,313"
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
# Reshape DataFrame from wide to long
df_long = pop_household_med_income.melt(
    id_vars=["Label (Grouping)"],  # Keep "Label (Grouping)" column
    var_name="County",  # Counties become the "County" column
    value_name="Value"  # The values stay under the "Value" column
)

# Pivot to get desired structure with counties as rows and stats as columns
df_pivoted = df_long.pivot_table(
    index="County",  # Use County as the index (rows)
    columns="Label (Grouping)",  # Use the statistics as columns
    values="Value",  # Use the corresponding value for each county/statistic
    aggfunc="first"  # Assuming no duplicates, use the first value
)

df_pivoted.head()

Label (Grouping),Median income (dollars),Total Households,Total Population
County,,,
"Alameda County, California","126,240","593,117","1,651,949"
"Alpine County, California","110,781",473,"1,695"
"Amador County, California","81,526","16,066","41,029"
"Butte County, California","68,574","82,345","209,470"
"Calaveras County, California","79,877","17,897","45,995"


In [25]:
pop_inc_households_df = df_pivoted.reset_index()
pop_inc_households_df.head()

Label (Grouping),County,Median income (dollars),Total Households,Total Population
0,"Alameda County, California","126,240","593,117","1,651,949"
1,"Alpine County, California","110,781",473,"1,695"
2,"Amador County, California","81,526","16,066","41,029"
3,"Butte County, California","68,574","82,345","209,470"
4,"Calaveras County, California","79,877","17,897","45,995"


In [26]:
pop_inc_households_df.rename(columns={
    "County": "county",
    "Median income (dollars)": "county_median_income",
    "Total Households": "county_total_households",
    "Total Population": "county_total_population"
}, inplace=True)
pop_inc_households_df.head()

Label (Grouping),county,county_median_income,county_total_households,county_total_population
0,"Alameda County, California","126,240","593,117","1,651,949"
1,"Alpine County, California","110,781",473,"1,695"
2,"Amador County, California","81,526","16,066","41,029"
3,"Butte County, California","68,574","82,345","209,470"
4,"Calaveras County, California","79,877","17,897","45,995"


In [27]:
# Remove ', California' from the 'county' column
pop_inc_households_df['county'] = pop_inc_households_df['county'].str.replace('County, California', '', regex=False)
pop_inc_households_df.head()

Label (Grouping),county,county_median_income,county_total_households,county_total_population
0,Alameda,"126,240","593,117","1,651,949"
1,Alpine,"110,781",473,"1,695"
2,Amador,"81,526","16,066","41,029"
3,Butte,"68,574","82,345","209,470"
4,Calaveras,"79,877","17,897","45,995"


In [28]:
pop_inc_households_df['county'] = pop_inc_households_df['county'].str.strip()
pop_inc_households_df.head()

Label (Grouping),county,county_median_income,county_total_households,county_total_population
0,Alameda,"126,240","593,117","1,651,949"
1,Alpine,"110,781",473,"1,695"
2,Amador,"81,526","16,066","41,029"
3,Butte,"68,574","82,345","209,470"
4,Calaveras,"79,877","17,897","45,995"


In [29]:
# This is the dataset that includes the race ACS data, redlining data, and geocoded HCD data

hcd_2023_ALL = pd.read_csv('/home/jovyan/bac/hcd_2023_ALL.csv')
hcd_2023_ALL.head()

/tmp/ipykernel_89/294905886.py:3: DtypeWarning: Columns (3,25,74,75,76,77,78,79,88,90,91,92,93,94,95,96,97,98,99,101) have mixed types. Specify dtype option on import or set low_memory=False.
  hcd_2023_ALL = pd.read_csv('/home/jovyan/bac/hcd_2023_ALL.csv')


,JURIS_NAME,CNTY_NAME,YEAR,PRIOR_APN,APN,STREET_ADDRESS,PROJECT_NAME,JURS_TRACKING_ID,UNIT_CAT,TENURE,...,inc_15k_to_25k,inc_25k_to_35k,inc_35k_to_50k,inc_50k_to_75k,inc_75k_to_100k,inc_100k_to_150k,inc_150k_to_200k,inc_200k_or_more,median_income,mean_income
0,ADELANTO,San Bernardino,2023,NaN,310310314,10352 San Marcos Ct,Alyssa Bottinelli,CAMINO-2023-1736,SFD,Owner,...,5.9,15.7,17.4,18.5,15.2,20.7,0.9,3.4,64613,75101.0
1,ADELANTO,San Bernardino,2023,NaN,310310315,10368 San Marcos Ct,Alyssa Bottinelli,CAMINO-2023-1737,SFD,Owner,...,5.9,15.7,17.4,18.5,15.2,20.7,0.9,3.4,64613,75101.0
2,ADELANTO,San Bernardino,2023,NaN,310310320,18786 Hampton Ln,Alyssa Bottinelli,CAMINO-2023-192,SFD,Owner,...,5.9,15.7,17.4,18.5,15.2,20.7,0.9,3.4,64613,75101.0
3,ADELANTO,San Bernardino,2023,NaN,310310320,18792 Hampton Ln,Alyssa Bottinelli,CAMINO-2023-191,SFD,Owner,...,5.9,15.7,17.4,18.5,15.2,20.7,0.9,3.4,64613,75101.0
4,ADELANTO,San Bernardino,2023,NaN,310310329,10353 San Marcos Ct,Alyssa Bottinelli,CAMINO-2023-1738,SFD,Owner,...,5.9,15.7,17.4,18.5,15.2,20.7,0.9,3.4,64613,75101.0


In [30]:
# Merge ratio and the other dataset
hcd_2023_ALL_ratio = pd.merge(hcd_2023_ALL, ratio_inc_pov, on=['GEO_ID'], how='left')
hcd_2023_ALL_ratio.head()
#hcd_2023_ALL_ratio.to_csv('hcd_2023_ALL_ratio.csv')

,JURIS_NAME,CNTY_NAME,YEAR,PRIOR_APN,APN,STREET_ADDRESS,PROJECT_NAME,JURS_TRACKING_ID,UNIT_CAT,TENURE,...,inc_75k_to_100k,inc_100k_to_150k,inc_150k_to_200k,inc_200k_or_more,median_income,mean_income,NAME,total_pop_y,ratio_inc_pov_under_one,p_ratio_inc_pov_under_one
0,ADELANTO,San Bernardino,2023,NaN,310310314,10352 San Marcos Ct,Alyssa Bottinelli,CAMINO-2023-1736,SFD,Owner,...,15.2,20.7,0.9,3.4,64613,75101.0,Census Tract 91.33; San Bernardino County; Cal...,1883.0,90.0,4.78
1,ADELANTO,San Bernardino,2023,NaN,310310315,10368 San Marcos Ct,Alyssa Bottinelli,CAMINO-2023-1737,SFD,Owner,...,15.2,20.7,0.9,3.4,64613,75101.0,Census Tract 91.33; San Bernardino County; Cal...,1883.0,90.0,4.78
2,ADELANTO,San Bernardino,2023,NaN,310310320,18786 Hampton Ln,Alyssa Bottinelli,CAMINO-2023-192,SFD,Owner,...,15.2,20.7,0.9,3.4,64613,75101.0,Census Tract 91.33; San Bernardino County; Cal...,1883.0,90.0,4.78
3,ADELANTO,San Bernardino,2023,NaN,310310320,18792 Hampton Ln,Alyssa Bottinelli,CAMINO-2023-191,SFD,Owner,...,15.2,20.7,0.9,3.4,64613,75101.0,Census Tract 91.33; San Bernardino County; Cal...,1883.0,90.0,4.78
4,ADELANTO,San Bernardino,2023,NaN,310310329,10353 San Marcos Ct,Alyssa Bottinelli,CAMINO-2023-1738,SFD,Owner,...,15.2,20.7,0.9,3.4,64613,75101.0,Census Tract 91.33; San Bernardino County; Cal...,1883.0,90.0,4.78


In [31]:
# Merge ratio and the other dataset
hcd_2023_ALL_ratio_pop = pd.merge(hcd_2023_ALL_ratio, df_pivoted, left_on=['County'], right_on=['County'], how='left')
hcd_2023_ALL_ratio_pop.head()
#hcd_2023_ALL_ratio_pop.to_csv('hcd_2023_ALL_ratio_pop.csv')

,JURIS_NAME,CNTY_NAME,YEAR,PRIOR_APN,APN,STREET_ADDRESS,PROJECT_NAME,JURS_TRACKING_ID,UNIT_CAT,TENURE,...,inc_200k_or_more,median_income,mean_income,NAME,total_pop_y,ratio_inc_pov_under_one,p_ratio_inc_pov_under_one,Median income (dollars),Total Households,Total Population
0,ADELANTO,San Bernardino,2023,NaN,310310314,10352 San Marcos Ct,Alyssa Bottinelli,CAMINO-2023-1736,SFD,Owner,...,3.4,64613,75101.0,Census Tract 91.33; San Bernardino County; Cal...,1883.0,90.0,4.78,NaN,NaN,NaN
1,ADELANTO,San Bernardino,2023,NaN,310310315,10368 San Marcos Ct,Alyssa Bottinelli,CAMINO-2023-1737,SFD,Owner,...,3.4,64613,75101.0,Census Tract 91.33; San Bernardino County; Cal...,1883.0,90.0,4.78,NaN,NaN,NaN
2,ADELANTO,San Bernardino,2023,NaN,310310320,18786 Hampton Ln,Alyssa Bottinelli,CAMINO-2023-192,SFD,Owner,...,3.4,64613,75101.0,Census Tract 91.33; San Bernardino County; Cal...,1883.0,90.0,4.78,NaN,NaN,NaN
3,ADELANTO,San Bernardino,2023,NaN,310310320,18792 Hampton Ln,Alyssa Bottinelli,CAMINO-2023-191,SFD,Owner,...,3.4,64613,75101.0,Census Tract 91.33; San Bernardino County; Cal...,1883.0,90.0,4.78,NaN,NaN,NaN
4,ADELANTO,San Bernardino,2023,NaN,310310329,10353 San Marcos Ct,Alyssa Bottinelli,CAMINO-2023-1738,SFD,Owner,...,3.4,64613,75101.0,Census Tract 91.33; San Bernardino County; Cal...,1883.0,90.0,4.78,NaN,NaN,NaN


# Race Shares Dashboard

In [69]:
hcd_2023_ALL

,JURIS_NAME,CNTY_NAME,YEAR,PRIOR_APN,APN,STREET_ADDRESS,PROJECT_NAME,JURS_TRACKING_ID,UNIT_CAT,TENURE,...,mean_income,CNTY_NAME_mode,CNTY_NAME_mode,CNTY_NAME_mode,CNTY_NAME_mode,CNTY_NAME_mode,CNTY_NAME_mode,CNTY_NAME_mode,most_frequent_county_x,most_frequent_county_y
0,ADELANTO,San Bernardino,2023,NaN,310310314,10352 San Marcos Ct,Alyssa Bottinelli,CAMINO-2023-1736,SFD,Owner,...,75101.0,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino
1,ADELANTO,San Bernardino,2023,NaN,310310315,10368 San Marcos Ct,Alyssa Bottinelli,CAMINO-2023-1737,SFD,Owner,...,75101.0,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino
2,ADELANTO,San Bernardino,2023,NaN,310310320,18786 Hampton Ln,Alyssa Bottinelli,CAMINO-2023-192,SFD,Owner,...,75101.0,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino
3,ADELANTO,San Bernardino,2023,NaN,310310320,18792 Hampton Ln,Alyssa Bottinelli,CAMINO-2023-191,SFD,Owner,...,75101.0,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino
4,ADELANTO,San Bernardino,2023,NaN,310310329,10353 San Marcos Ct,Alyssa Bottinelli,CAMINO-2023-1738,SFD,Owner,...,75101.0,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123755,YUCCA VALLEY,San Bernardino,2023,NaN,0598-601-10,58855 MEREDITH CT,NaN,2021-2239,SFD,Owner,...,83284.0,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino
123756,YUCCA VALLEY,San Bernardino,2023,NaN,0598-601-12,58871 MEREDITH CT,NaN,2021-2266,SFD,Owner,...,83284.0,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino
123757,YUCCA VALLEY,San Bernardino,2023,NaN,0601-021-43,57750 PAXTON RD,NaN,2023-333,SFD,Owner,...,83284.0,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino
123758,YUCCA VALLEY,San Bernardino,2023,NaN,0601-081-66,59045 WILCOX LN,NaN,2022-905,SFD,Owner,...,83284.0,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino,San Bernardino


In [70]:
# Convert 'majority_poc_flag' to numeric type
hcd_2023_ALL['majority_poc_flag'] = pd.to_numeric(hcd_2023_ALL['majority_poc_flag'], errors='coerce')

In [71]:
# Step 1: Assign the most frequently occurring county for each tract
tract_county_mode = (
    hcd_2023_ALL.groupby('TRACTCE')['CNTY_NAME']
    .apply(lambda x: x.value_counts().idxmax())  # Find the most frequent county
    .reset_index()
    .rename(columns={'CNTY_NAME': 'most_frequent_county'})  # Rename column for clarity
)

# Merge back to the original dataset
hcd_2023_ALL = hcd_2023_ALL.merge(tract_county_mode, on='TRACTCE', how='left')

# Step 2: Aggregate to the tract level
tract_level = hcd_2023_ALL.groupby('TRACTCE', as_index=False).agg(
    county=('most_frequent_county', 'first'),  # Use assigned county
    maj_poc_flag=('majority_poc_flag', 'max')  # If any record in the tract has maj_poc_flag = 1, assign 1
).drop_duplicates(subset=['TRACTCE'])

# Step 3: Aggregate to the county level
county_stats = tract_level.groupby('county', as_index=False).agg(
    total_tracts=('TRACTCE', 'count'),
    maj_poc_tracts=('maj_poc_flag', 'sum')  # Count tracts where maj_poc_flag = 1
)

# Step 4: Calculate the share of majority POC tracts
county_stats['share_maj_poc_tracts'] = (county_stats['maj_poc_tracts'] / county_stats['total_tracts'] * 100).round(1)

# Display the first few rows
county_stats.head(20)


,county,total_tracts,maj_poc_tracts,share_maj_poc_tracts
0,Alameda,302,244,80.8
1,Amador,4,3,75.0
2,Butte,20,19,95.0
3,Calaveras,15,7,46.7
4,Colusa,1,1,100.0
5,Contra Costa,214,129,60.3
6,Del Norte,3,2,66.7
7,El Dorado,49,11,22.4
8,Fresno,101,82,81.2
9,Humboldt,4,3,75.0


In [50]:
# SHARE OF TRACTS THAT ARE MAJORITY POC

# Aggregate to the tract level
tract_level = hcd_2023_ALL.groupby('GEO_ID').agg(
    county=('CNTY_NAME', 'first'),  # Assuming each tract belongs to one county
    maj_poc_flag=('majority_poc_flag', 'max')  # If any address in the tract has maj_poc_flag = 1, assign 1
).reset_index()

# Step 2: Aggregate to the county level
county_stats = tract_level.groupby('county').agg(
    total_tracts=('GEO_ID', 'count'),
    maj_poc_tracts=('maj_poc_flag', 'sum')  # Count tracts with maj_poc_flag = 1
).reset_index()

# Step 3: Calculate the share of tracts with maj_poc_flag = 1
county_stats['share_maj_poc_tracts'] = (county_stats['maj_poc_tracts'] / county_stats['total_tracts'] * 100).round(1)

county_stats.head(20)

,county,total_tracts,maj_poc_tracts,share_maj_poc_tracts
0,Alameda,333,270,81.1
1,Alpine,1,0,0.0
2,Amador,16,5,31.2
3,Butte,110,29,26.4
4,Calaveras,72,25,34.7
5,Colusa,3,3,100.0
6,Contra Costa,213,130,61.0
7,Del Norte,14,5,35.7
8,El Dorado,52,3,5.8
9,Fresno,186,156,83.9


In [33]:
# SHARE OF ADUS PERMITTED IN MAJORITY POC AREAS

# Ensure 'majority_poc_flag' and 'no_building_permits' are numeric
hcd_2023_ALL['majority_poc_flag'] = pd.to_numeric(hcd_2023_ALL['majority_poc_flag'], errors='coerce').fillna(0).astype(int)
hcd_2023_ALL['NO_BUILDING_PERMITS'] = pd.to_numeric(hcd_2023_ALL['NO_BUILDING_PERMITS'], errors='coerce').fillna(0)

# Step 1: Aggregate total ADUs permitted by county
county_adus = hcd_2023_ALL.groupby('CNTY_NAME').agg(
    total_adus=('NO_BUILDING_PERMITS', 'sum'),
    adus_in_maj_poc=('NO_BUILDING_PERMITS', lambda x: x[hcd_2023_ALL['majority_poc_flag'] == 1].sum())
).reset_index()

# Step 2: Calculate the share of ADUs in majority POC areas by county (as percentage, rounded to 1 decimal place)
county_adus['share_adus_maj_poc'] = (county_adus['adus_in_maj_poc'] / county_adus['total_adus'] * 100).round(1)

county_adus.head()

,CNTY_NAME,total_adus,adus_in_maj_poc,share_adus_maj_poc
0,Alameda,5166,4679,90.6
1,Alpine,10,0,0.0
2,Amador,127,7,5.5
3,Butte,1029,137,13.3
4,Calaveras,331,58,17.5


In [34]:
# Merge pov and income datasets
race_shares_2023 = pd.merge(county_stats, county_adus, left_on=['county'], right_on=['CNTY_NAME'], how='inner')
race_shares_2023.head()
#race_shares_2023.to_csv('race_shares_2023.csv')

,county,total_tracts,maj_poc_tracts,share_maj_poc_tracts,CNTY_NAME,total_adus,adus_in_maj_poc,share_adus_maj_poc
0,Alameda,333,270.0,81.1,Alameda,5166,4679,90.6
1,Alpine,1,0.0,0.0,Alpine,10,0,0.0
2,Amador,16,5.0,31.2,Amador,127,7,5.5
3,Butte,110,29.0,26.4,Butte,1029,137,13.3
4,Calaveras,72,25.0,34.7,Calaveras,331,58,17.5


In [35]:
# Calculate the total ADUs permitted in California
total_adus_california = race_shares_2023['total_adus'].sum()

# Calculate the total ADUs permitted in Majority POC tracts in California
total_adus_maj_poc = race_shares_2023['adus_in_maj_poc'].sum()

# Compute the percentage of ADUs permitted in Majority POC tracts
percentage_adus_maj_poc = (total_adus_maj_poc / total_adus_california) * 100 if total_adus_california > 0 else 0

# Print the result
print(f"Percentage of ADUs permitted in Majority POC tracts in California: {percentage_adus_maj_poc:.2f}%")


Percentage of ADUs permitted in Majority POC tracts in California: 69.19%


In [36]:
# Calculate the total number of tracts in California
total_tracts_california = race_shares_2023['total_tracts'].sum()

# Calculate the total number of Majority POC tracts in California
total_maj_poc_tracts = race_shares_2023['maj_poc_tracts'].sum()

# Compute the percentage of tracts that are Majority POC
percentage_maj_poc_tracts = (total_maj_poc_tracts / total_tracts_california) * 100 if total_tracts_california > 0 else 0

# Print the result
print(f"Percentage of tracts that are Majority POC in California: {percentage_maj_poc_tracts:.2f}%")


Percentage of tracts that are Majority POC in California: 66.61%


In [37]:
# Define Bay Area counties
bay_area_counties = ['Alameda', 'Contra Costa', 'Marin', 'Napa', 'San Francisco', 
                     'San Mateo', 'Santa Clara', 'Solano', 'Sonoma']

# Filter for Bay Area region
bay_area_df = race_shares_2023[race_shares_2023['county'].isin(bay_area_counties)]  

# Calculate the total ADUs permitted in the Bay Area
total_adus_bay_area = bay_area_df['total_adus'].sum()

# Calculate the total ADUs permitted in Majority POC tracts in the Bay Area
total_adus_maj_poc_bay_area = bay_area_df['adus_in_maj_poc'].sum()

# Compute the percentage of ADUs permitted in Majority POC tracts in the Bay Area
percentage_adus_maj_poc_bay_area = (total_adus_maj_poc_bay_area / total_adus_bay_area) * 100 if total_adus_bay_area > 0 else 0

# Calculate the total number of tracts in the Bay Area
total_tracts_bay_area = bay_area_df['total_tracts'].sum()

# Calculate the total number of Majority POC tracts in the Bay Area
total_maj_poc_tracts_bay_area = bay_area_df['maj_poc_tracts'].sum()

# Compute the percentage of tracts that are Majority POC in the Bay Area
percentage_maj_poc_tracts_bay_area = (total_maj_poc_tracts_bay_area / total_tracts_bay_area) * 100 if total_tracts_bay_area > 0 else 0

# Print the results
print(f"Percentage of ADUs permitted in Majority POC tracts in the Bay Area: {percentage_adus_maj_poc_bay_area:.2f}%")
print(f"Percentage of tracts that are Majority POC in the Bay Area: {percentage_maj_poc_tracts_bay_area:.2f}%")


Percentage of ADUs permitted in Majority POC tracts in the Bay Area: 73.35%
Percentage of tracts that are Majority POC in the Bay Area: 69.40%


# Doing the same for below poverty and below median income

In [38]:
hcd_2023_ALL

,JURIS_NAME,CNTY_NAME,YEAR,PRIOR_APN,APN,STREET_ADDRESS,PROJECT_NAME,JURS_TRACKING_ID,UNIT_CAT,TENURE,...,inc_15k_to_25k,inc_25k_to_35k,inc_35k_to_50k,inc_50k_to_75k,inc_75k_to_100k,inc_100k_to_150k,inc_150k_to_200k,inc_200k_or_more,median_income,mean_income
0,ADELANTO,San Bernardino,2023,NaN,310310314,10352 San Marcos Ct,Alyssa Bottinelli,CAMINO-2023-1736,SFD,Owner,...,5.9,15.7,17.4,18.5,15.2,20.7,0.9,3.4,64613,75101.0
1,ADELANTO,San Bernardino,2023,NaN,310310315,10368 San Marcos Ct,Alyssa Bottinelli,CAMINO-2023-1737,SFD,Owner,...,5.9,15.7,17.4,18.5,15.2,20.7,0.9,3.4,64613,75101.0
2,ADELANTO,San Bernardino,2023,NaN,310310320,18786 Hampton Ln,Alyssa Bottinelli,CAMINO-2023-192,SFD,Owner,...,5.9,15.7,17.4,18.5,15.2,20.7,0.9,3.4,64613,75101.0
3,ADELANTO,San Bernardino,2023,NaN,310310320,18792 Hampton Ln,Alyssa Bottinelli,CAMINO-2023-191,SFD,Owner,...,5.9,15.7,17.4,18.5,15.2,20.7,0.9,3.4,64613,75101.0
4,ADELANTO,San Bernardino,2023,NaN,310310329,10353 San Marcos Ct,Alyssa Bottinelli,CAMINO-2023-1738,SFD,Owner,...,5.9,15.7,17.4,18.5,15.2,20.7,0.9,3.4,64613,75101.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123755,YUCCA VALLEY,San Bernardino,2023,NaN,0598-601-10,58855 MEREDITH CT,NaN,2021-2239,SFD,Owner,...,10.0,9.1,21.4,22.6,5.7,11.4,12.6,6.6,57524,83284.0
123756,YUCCA VALLEY,San Bernardino,2023,NaN,0598-601-12,58871 MEREDITH CT,NaN,2021-2266,SFD,Owner,...,10.0,9.1,21.4,22.6,5.7,11.4,12.6,6.6,57524,83284.0
123757,YUCCA VALLEY,San Bernardino,2023,NaN,0601-021-43,57750 PAXTON RD,NaN,2023-333,SFD,Owner,...,10.0,9.1,21.4,22.6,5.7,11.4,12.6,6.6,57524,83284.0
123758,YUCCA VALLEY,San Bernardino,2023,NaN,0601-081-66,59045 WILCOX LN,NaN,2022-905,SFD,Owner,...,10.0,9.1,21.4,22.6,5.7,11.4,12.6,6.6,57524,83284.0


In [39]:
# Drop columns that start with 'inc_'
hcd_2023_ALL_INCOME = hcd_2023_ALL.drop(columns=[col for col in hcd_2023_ALL.columns if col.startswith("inc_")])
hcd_2023_ALL_INCOME.head()

,JURIS_NAME,CNTY_NAME,YEAR,PRIOR_APN,APN,STREET_ADDRESS,PROJECT_NAME,JURS_TRACKING_ID,UNIT_CAT,TENURE,...,tot_tworaces,GEO_ID,NAME_y,total_pop,pop_below_pov_level,p_below_pov_level,tot_households,less_than_10k,median_income,mean_income
0,ADELANTO,San Bernardino,2023,NaN,310310314,10352 San Marcos Ct,Alyssa Bottinelli,CAMINO-2023-1736,SFD,Owner,...,0.0,6.071009e+09,Census Tract 91.33; San Bernardino County; Cal...,5781.0,343.0,5.9,1601.0,0.8,64613,75101.0
1,ADELANTO,San Bernardino,2023,NaN,310310315,10368 San Marcos Ct,Alyssa Bottinelli,CAMINO-2023-1737,SFD,Owner,...,0.0,6.071009e+09,Census Tract 91.33; San Bernardino County; Cal...,5781.0,343.0,5.9,1601.0,0.8,64613,75101.0
2,ADELANTO,San Bernardino,2023,NaN,310310320,18786 Hampton Ln,Alyssa Bottinelli,CAMINO-2023-192,SFD,Owner,...,0.0,6.071009e+09,Census Tract 91.33; San Bernardino County; Cal...,5781.0,343.0,5.9,1601.0,0.8,64613,75101.0
3,ADELANTO,San Bernardino,2023,NaN,310310320,18792 Hampton Ln,Alyssa Bottinelli,CAMINO-2023-191,SFD,Owner,...,0.0,6.071009e+09,Census Tract 91.33; San Bernardino County; Cal...,5781.0,343.0,5.9,1601.0,0.8,64613,75101.0
4,ADELANTO,San Bernardino,2023,NaN,310310329,10353 San Marcos Ct,Alyssa Bottinelli,CAMINO-2023-1738,SFD,Owner,...,0.0,6.071009e+09,Census Tract 91.33; San Bernardino County; Cal...,5781.0,343.0,5.9,1601.0,0.8,64613,75101.0


In [40]:
hcd_2023_ALL_INCOME_2 = pd.merge(hcd_2023_ALL_INCOME, pop_inc_households_df, left_on='CNTY_NAME', right_on='county', how='inner')
hcd_2023_ALL_INCOME_2

,JURIS_NAME,CNTY_NAME,YEAR,PRIOR_APN,APN,STREET_ADDRESS,PROJECT_NAME,JURS_TRACKING_ID,UNIT_CAT,TENURE,...,pop_below_pov_level,p_below_pov_level,tot_households,less_than_10k,median_income,mean_income,county,county_median_income,county_total_households,county_total_population
0,ADELANTO,San Bernardino,2023,NaN,310310314,10352 San Marcos Ct,Alyssa Bottinelli,CAMINO-2023-1736,SFD,Owner,...,343.0,5.9,1601.0,0.8,64613,75101.0,San Bernardino,"82,184","668,004","2,187,816"
1,ADELANTO,San Bernardino,2023,NaN,310310315,10368 San Marcos Ct,Alyssa Bottinelli,CAMINO-2023-1737,SFD,Owner,...,343.0,5.9,1601.0,0.8,64613,75101.0,San Bernardino,"82,184","668,004","2,187,816"
2,ADELANTO,San Bernardino,2023,NaN,310310320,18786 Hampton Ln,Alyssa Bottinelli,CAMINO-2023-192,SFD,Owner,...,343.0,5.9,1601.0,0.8,64613,75101.0,San Bernardino,"82,184","668,004","2,187,816"
3,ADELANTO,San Bernardino,2023,NaN,310310320,18792 Hampton Ln,Alyssa Bottinelli,CAMINO-2023-191,SFD,Owner,...,343.0,5.9,1601.0,0.8,64613,75101.0,San Bernardino,"82,184","668,004","2,187,816"
4,ADELANTO,San Bernardino,2023,NaN,310310329,10353 San Marcos Ct,Alyssa Bottinelli,CAMINO-2023-1738,SFD,Owner,...,343.0,5.9,1601.0,0.8,64613,75101.0,San Bernardino,"82,184","668,004","2,187,816"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123755,TUOLUMNE COUNTY,Tuolumne,2023,NaN,85470006000,21485 PASEO DE LOS PORTALES,NaN,HB-23-2390,SFD,Owner,...,164.0,5.4,1254.0,1.1,98152,104879.0,Tuolumne,"72,259","22,809","54,873"
123756,TUOLUMNE COUNTY,Tuolumne,2023,NaN,88050026000,"17809 RUBY RD, TUOLUMNE, CA 95379",NaN,B2023-00849,MH,Owner,...,178.0,3.0,2257.0,1.3,78292,91904.0,Tuolumne,"72,259","22,809","54,873"
123757,TUOLUMNE COUNTY,Tuolumne,2023,NaN,90110003000,12454 CASSARETTO CT,NaN,HB-23-2384,SFD,Owner,...,203.0,8.8,984.0,1.9,66536,102401.0,Tuolumne,"72,259","22,809","54,873"
123758,TUOLUMNE COUNTY,Tuolumne,2023,NaN,90250062000,19288 OAK GROVE CIRCLE,NaN,HB-21-935,SFD,Owner,...,203.0,8.8,984.0,1.9,66536,102401.0,Tuolumne,"72,259","22,809","54,873"


In [41]:
# Calculate STATE STUFF

# Define California's median income (update with the correct value if needed)
california_median_income = 89870  # Ensure it's an integer, not a tuple

# Filter for census tracts where median income is below the state median
below_median_income_tracts = hcd_2023_ALL_INCOME_2[hcd_2023_ALL_INCOME_2['median_income'] < california_median_income]

# Calculate the total ADUs permitted in those tracts
adus_in_below_median_tracts = below_median_income_tracts['NO_BUILDING_PERMITS'].sum()

# Calculate the total ADUs permitted in all tracts
total_adus_permitted = hcd_2023_ALL_INCOME_2['NO_BUILDING_PERMITS'].sum()  # Use the correct dataframe

# Compute the percentage
percentage_below_median = (adus_in_below_median_tracts / total_adus_permitted) * 100 if total_adus_permitted > 0 else 0

# Display the result
print(f"Percentage of ADUs permitted in tracts below California’s median income: {percentage_below_median:.2f}%")


TypeError: '<' not supported between instances of 'str' and 'int'

In [ ]:
# Filter DataFrame for Bay Area counties
bay_area_df = hcd_2023_ALL_INCOME_2[hcd_2023_ALL_INCOME_2['Region'] == 'Bay Area']

# Calculate the median income for Bay Area counties
median_income_bay_area = bay_area_df['county_median_income'].median()

median_income_bay_area

In [ ]:
# Calculate Bay Area-specific percentage of ADUs in low-income tracts

# Define California's median income (update with the correct value if needed)
bay_area_median_income = 126240  # Ensure it's an integer, not a tuple

# Filter for Bay Area region
bay_area_data = hcd_2023_ALL_INCOME_2[hcd_2023_ALL_INCOME_2['Region'] == 'Bay Area']

# Filter for census tracts where median income is below the state median
below_median_income_tracts = bay_area_data[bay_area_data['median_income'] < bay_area_median_income]

# Calculate the total ADUs permitted in those tracts
adus_in_below_median_tracts = below_median_income_tracts['NO_BUILDING_PERMITS'].sum()

# Calculate the total ADUs permitted in all tracts in the Bay Area
total_adus_permitted = bay_area_data['NO_BUILDING_PERMITS'].sum()  

# Compute the percentage
percentage_below_median = (adus_in_below_median_tracts / total_adus_permitted) * 100 if total_adus_permitted > 0 else 0

# Display the result
print(f"Percentage of ADUs permitted in Bay Area tracts below California’s median income: {percentage_below_median:.2f}%")


In [ ]:
# Ensure numeric conversion
hcd_2023_ALL_INCOME_2['county_median_income'] = hcd_2023_ALL_INCOME_2['county_median_income'].astype(str).str.replace(',', '', regex=True)
hcd_2023_ALL_INCOME_2['county_median_income'] = pd.to_numeric(hcd_2023_ALL_INCOME_2['county_median_income'], errors='coerce')
hcd_2023_ALL_INCOME_2['median_income'] = pd.to_numeric(hcd_2023_ALL_INCOME_2['median_income'], errors='coerce')

# Step 1: Create a flag for tracts below the COUNTY MEDIAN INCOME
hcd_2023_ALL_INCOME_2['below_county_median_income_flag'] = (
    hcd_2023_ALL_INCOME_2['median_income'] < hcd_2023_ALL_INCOME_2['county_median_income']
).astype(int)

# Check if the flag is being set correctly
print("Unique values in below_county_median_income_flag:", hcd_2023_ALL_INCOME_2['below_county_median_income_flag'].value_counts())

# Step 2: Compute share of tracts below COUNTY median income by county
tract_level_income = hcd_2023_ALL_INCOME_2.groupby('GEO_ID').agg(
    county=('CNTY_NAME', 'first'),  
    below_county_median_income_flag=('below_county_median_income_flag', 'max')
).reset_index()

# Step 3: Compute county-level statistics
county_income_stats = tract_level_income.groupby('county').agg(
    total_tracts=('GEO_ID', 'count'),
    below_county_median_income_tracts=('below_county_median_income_flag', 'sum')
).reset_index()

# Step 4: Calculate the share of tracts below county median income
county_income_stats['share_below_county_median_income_tracts'] = (
    county_income_stats['below_county_median_income_tracts'] / county_income_stats['total_tracts'] * 100
).round(1)

# Display result
county_income_stats.head()

In [ ]:
# Step 5: Compute the share of ADUs permitted in below-county-median-income areas by county
hcd_2023_ALL_INCOME_2['NO_BUILDING_PERMITS'] = pd.to_numeric(hcd_2023_ALL_INCOME_2['NO_BUILDING_PERMITS'], errors='coerce').fillna(0)

# **Fix: Pre-compute ADUs in below-median-income areas before grouping**
hcd_2023_ALL_INCOME_2['adus_in_below_county_median_income'] = (
    hcd_2023_ALL_INCOME_2['NO_BUILDING_PERMITS'] * hcd_2023_ALL_INCOME_2['below_county_median_income_flag']
)

# Group by county and compute totals
county_adus_income = hcd_2023_ALL_INCOME_2.groupby('CNTY_NAME').agg(
    total_adus=('NO_BUILDING_PERMITS', 'sum'),
    adus_in_below_county_median_income=('adus_in_below_county_median_income', 'sum')
).reset_index()

# Step 6: Calculate the share of ADUs permitted in below-county-median-income areas
county_adus_income['share_adus_below_county_median_income'] = (
    county_adus_income['adus_in_below_county_median_income'] / county_adus_income['total_adus'] * 100
).round(1)

county_adus_income.head()

In [ ]:
# Merge both results into one DataFrame
county_income_shares_final = pd.merge(county_income_stats, county_adus_income, left_on='county', right_on='CNTY_NAME', how='left')

In [ ]:
# Merge back the median income data
county_income_shares_final = pd.merge(county_income_shares_final, pop_inc_households_df, 
                                      left_on='CNTY_NAME', right_on='county', how='inner')

county_income_shares_final
county_income_shares_final.to_csv('county_income_shares_final_usingmedincomebycounty.csv')

## Now poverty

In [ ]:
# California poverty rate was 18.9% in 2023

# Ensure 'p_below_pov_level' is numeric
hcd_2023_ALL['p_below_pov_level'] = pd.to_numeric(hcd_2023_ALL['p_below_pov_level'], errors='coerce')

# Create a flag for tracts where poverty rate is above California's poverty rate (18.9%)
california_poverty_rate = 18.9
hcd_2023_ALL['below_state_poverty_flag'] = (hcd_2023_ALL['p_below_pov_level'] > california_poverty_rate).astype(int)

# Compute share of tracts above the state poverty rate by county
tract_level_poverty = hcd_2023_ALL.groupby('GEO_ID').agg(
    county=('CNTY_NAME', 'first'),  # Each tract belongs to one county
    below_state_poverty_flag=('below_state_poverty_flag', 'max')  # If any address in the tract is above state poverty rate, assign 1
).reset_index()

county_poverty_stats = tract_level_poverty.groupby('county').agg(
    total_tracts=('GEO_ID', 'count'),
    high_poverty_tracts=('below_state_poverty_flag', 'sum')  # Count tracts above the state poverty rate
).reset_index()

# Calculate the share of tracts above the state poverty rate by county
county_poverty_stats['share_high_poverty_tracts'] = (
    county_poverty_stats['high_poverty_tracts'] / county_poverty_stats['total_tracts'] * 100
).round(1)

county_poverty_stats.head()

In [ ]:
# Compute the share of ADUs permitted in high-poverty areas by county
hcd_2023_ALL['NO_BUILDING_PERMITS'] = pd.to_numeric(hcd_2023_ALL['NO_BUILDING_PERMITS'], errors='coerce').fillna(0)

county_adus_poverty = hcd_2023_ALL.groupby('CNTY_NAME').agg(
    total_adus=('NO_BUILDING_PERMITS', 'sum'),
    adus_in_high_poverty=('NO_BUILDING_PERMITS', lambda x: x[hcd_2023_ALL['below_state_poverty_flag'] == 1].sum())
).reset_index()

# Calculate the share of ADUs permitted in high-poverty areas
county_adus_poverty['share_adus_high_poverty'] = (
    county_adus_poverty['adus_in_high_poverty'] / county_adus_poverty['total_adus'] * 100
).round(1)

county_adus_poverty.head()


In [ ]:
# Merge both results into one DataFrame
county_final_poverty = pd.merge(county_poverty_stats, county_adus_poverty, left_on='county', right_on='CNTY_NAME', how='left')
county_final_poverty.head()
#county_final_poverty.to_csv('county_final_poverty.csv')

# By Income/Poverty Ratio

In [ ]:
# Ensure necessary columns are numeric
hcd_2023_ALL_ratio_pop['p_ratio_inc_pov_under_one'] = pd.to_numeric(hcd_2023_ALL_ratio_pop['p_ratio_inc_pov_under_one'], errors='coerce')
hcd_2023_ALL_ratio_pop['NO_BUILDING_PERMITS'] = pd.to_numeric(hcd_2023_ALL_ratio_pop['NO_BUILDING_PERMITS'], errors='coerce').fillna(0)

# Compute tract-level share with ratios < 1 and group by county
tract_level_poverty = hcd_2023_ALL_ratio_pop.groupby('GEO_ID').agg(
    county=('CNTY_NAME', 'first'),
    avg_p_ratio_inc_pov_under_one=('p_ratio_inc_pov_under_one', 'mean')  # Avg percentage of people in poverty per tract
).reset_index()

# Compute total tracts and weighted share by county
county_poverty_stats = tract_level_poverty.groupby('county').agg(
    total_tracts=('GEO_ID', 'count'),
    avg_poverty_share=('avg_p_ratio_inc_pov_under_one', 'mean')  # County-level average share of people in poverty
).reset_index()

# Compute the share of ADUs permitted in areas with a high percentage of income-poverty ratio < 1
hcd_2023_ALL_ratio_pop['weighted_adus_in_poverty'] = hcd_2023_ALL_ratio_pop['NO_BUILDING_PERMITS'] * (hcd_2023_ALL_ratio_pop['p_ratio_inc_pov_under_one'] / 100)

county_adus_poverty = hcd_2023_ALL_ratio_pop.groupby('CNTY_NAME').agg(
    total_adus=('NO_BUILDING_PERMITS', 'sum'),
    weighted_adus_in_poverty=('weighted_adus_in_poverty', 'sum')  # Sum of ADUs weighted by tract's poverty percentage
).reset_index()

# Calculate the share of ADUs in tracts with high poverty percentages
county_adus_poverty['share_adus_in_poverty'] = (
    county_adus_poverty['weighted_adus_in_poverty'] / county_adus_poverty['total_adus'] * 100
).round(1)

# Merge both results into one DataFrame
county_final_poverty = pd.merge(county_poverty_stats, county_adus_poverty, left_on='county', right_on='CNTY_NAME', how='left')
county_final_poverty.head()
#county_final_poverty.to_csv('ratio_county_final_poverty.csv')

# By Redlining Grade

In [ ]:
hcd_2023_ALL

In [ ]:
import pandas as pd

# Filter for relevant grades
hcd_2023_ALL = hcd_2023_ALL[hcd_2023_ALL['grade'].isin(['A', 'B', 'C', 'D'])].copy()

# Ensure necessary columns are properly formatted
hcd_2023_ALL['grade'] = hcd_2023_ALL['grade'].astype(str)  # Ensure grade is a string
hcd_2023_ALL['NO_BUILDING_PERMITS'] = pd.to_numeric(hcd_2023_ALL['NO_BUILDING_PERMITS'], errors='coerce').fillna(0)

# Compute total number of developments (rows) per city
city_adus_total = hcd_2023_ALL.groupby('JURIS_NAME', as_index=False).agg(
    total_adus=('JURIS_NAME', 'count')  # Count number of rows (developments) per city
)

# Compute ADUs by redlining grade per city
city_adus_by_grade = hcd_2023_ALL.groupby(['JURIS_NAME', 'grade'], as_index=False).agg(
    adus_in_grade=('JURIS_NAME', 'count')  # Count number of rows (developments) per grade
)

# Merge with total ADUs per city
city_adus_by_grade = city_adus_by_grade.merge(city_adus_total, on='JURIS_NAME', how='left')

# Calculate share of ADUs in each redlining grade per city based on row count
city_adus_by_grade['share_adus_in_grade'] = (
    city_adus_by_grade['adus_in_grade'] / city_adus_by_grade['total_adus'] * 100
).round(1)

# Pivot the data to show shares for A, B, C, D as separate columns per city
city_adus_pivot = city_adus_by_grade.pivot(index='JURIS_NAME', columns='grade', values='share_adus_in_grade').reset_index()

# Rename columns for clarity (fill missing values with 0)
city_adus_pivot = city_adus_pivot.rename(columns=lambda x: f"share_adus_{x}" if x in ['A', 'B', 'C', 'D'] else x).fillna(0)

In [ ]:
# Remove rows where all share columns (A, B, C, D) are 0
share_columns = ['share_adus_A', 'share_adus_B', 'share_adus_C', 'share_adus_D']
city_adus_pivot = city_adus_pivot.loc[city_adus_pivot[share_columns].any(axis=1)]
city_adus_pivot

In [ ]:
# Convert from wide to long format for Tableau
redlined_city_adus_long = city_adus_pivot.melt(
    id_vars=["JURIS_NAME"],  # Keep city name
    value_vars=["share_adus_A", "share_adus_B", "share_adus_C", "share_adus_D"],  # Pivot these columns
    var_name="Redlining_Grade",  # New column for A, B, C, D, NA categories
    value_name="ADU_Share"  # New column for ADU share values
)

# Rename redlining grade values for clarity
redlined_city_adus_long['Redlining_Grade'] = redlined_city_adus_long['Redlining_Grade'].str.replace("share_adus_", "")
#redlined_city_adus_long['Redlining_Grade'] = redlined_city_adus_long['Redlining_Grade'].replace({"nan": "NA"})  # Rename NaN column

# Save the cleaned dataset for Tableau
redlined_city_adus_long
#redlined_city_adus_long.to_csv('city_adus_pivot_gradesonly_normalized.csv', index=False)

# Redlined not normalized

In [ ]:
# Ensure necessary columns are properly formatted
hcd_2023_ALL['grade'] = hcd_2023_ALL['grade'].astype(str)  # Ensure grade is a string
hcd_2023_ALL['NO_BUILDING_PERMITS'] = pd.to_numeric(hcd_2023_ALL['NO_BUILDING_PERMITS'], errors='coerce').fillna(0)

# Compute total ADUs by city
city_adus_total = hcd_2023_ALL.groupby('JURIS_NAME').agg(
    total_adus=('NO_BUILDING_PERMITS', 'sum')
).reset_index()

# Compute ADUs by redlining grade per county
city_adus_by_grade = hcd_2023_ALL.groupby(['JURIS_NAME', 'grade']).agg(
    adus_in_grade=('NO_BUILDING_PERMITS', 'sum')
).reset_index()

# Merge with total ADUs to calculate shares
city_adus_by_grade = city_adus_by_grade.merge(city_adus_total, on='JURIS_NAME', how='left')

# Calculate share of ADUs in each redlining grade per city
city_adus_by_grade['share_adus_in_grade'] = (
    city_adus_by_grade['adus_in_grade'] / city_adus_by_grade['total_adus'] * 100
).round(1)

# Pivot the data to show shares for A, B, C, D as separate columns per county
city_adus_pivot = city_adus_by_grade.pivot(index='JURIS_NAME', columns='grade', values='share_adus_in_grade').reset_index()

# Rename columns for clarity
city_adus_pivot = city_adus_pivot.rename(columns={
    'A': 'share_adus_A',
    'B': 'share_adus_B',
    'C': 'share_adus_C',
    'D': 'share_adus_D'
})

# Remove counties with any NaN values
city_adus_pivot = city_adus_pivot.dropna()

# Display the final results
city_adus_pivot
#county_adus_pivot.to_csv('county_adus_pivot.csv')

In [ ]:
# Convert from wide to long format for Tableau
redlined_city_adus_long = city_adus_pivot.melt(
    id_vars=["JURIS_NAME"],  # Keep city name
    value_vars=["share_adus_A", "share_adus_B", "share_adus_C", "share_adus_D", "nan"],  # Pivot these columns
    var_name="Redlining_Grade",  # New column for A, B, C, D, NA categories
    value_name="ADU_Share"  # New column for ADU share values
)

# Rename redlining grade values for clarity
redlined_city_adus_long['Redlining_Grade'] = redlined_city_adus_long['Redlining_Grade'].str.replace("share_adus_", "")
redlined_city_adus_long['Redlining_Grade'] = redlined_city_adus_long['Redlining_Grade'].replace({"nan": "NA"})  # Rename NaN column

# Save the cleaned dataset for Tableau
redlined_city_adus_long.head()
#redlined_city_adus_long.to_csv('redlined_city_adus_long.csv', index=False)


# Redlining for 2018

In [ ]:
redlining_2018 = pd.read_csv('/home/jovyan/bac/housing_permits_2018_withredlining.csv')
redlining_2018.head()

In [ ]:
# Filter for relevant grades
redlining_2018 = redlining_2018[redlining_2018['grade'].isin(['A', 'B', 'C', 'D'])].copy()
redlining_2018.head()

In [ ]:
redlining_2018 = redlining_2018.iloc[:, 3:]
redlining_2018

In [ ]:
import pandas as pd

# Ensure necessary columns are properly formatted
redlining_2018['grade'] = redlining_2018['grade'].astype(str)  # Ensure grade is a string
redlining_2018['Total_Units'] = pd.to_numeric(redlining_2018['Total_Units'], errors='coerce').fillna(0)

# Compute total number of developments (rows) per city
city_adus_total = redlining_2018.groupby('city', as_index=False).agg(
    total_adus=('city', 'count')  # Count number of rows (developments) per city
)

# Compute ADUs by redlining grade per city
city_adus_by_grade = redlining_2018.groupby(['city', 'grade'], as_index=False).agg(
    adus_in_grade=('city', 'count')  # Count number of rows (developments) per grade
)

# Merge with total ADUs per city
city_adus_by_grade = city_adus_by_grade.merge(city_adus_total, on='city', how='left')

# Calculate share of ADUs in each redlining grade per city based on row count
city_adus_by_grade['share_adus_in_grade'] = (
    city_adus_by_grade['adus_in_grade'] / city_adus_by_grade['total_adus'] * 100
).round(1)

# Pivot the data to show shares for A, B, C, D as separate columns per city
city_adus_pivot = city_adus_by_grade.pivot(index='city', columns='grade', values='share_adus_in_grade').reset_index()

# Rename columns for clarity (fill missing values with 0)
city_adus_pivot = city_adus_pivot.rename(
    columns=lambda x: f"share_adus_{str(x)}" if x in ['A', 'B', 'C', 'D'] else x
).fillna(0)

# Display the cleaned DataFrame
city_adus_pivot

In [ ]:
# Remove rows where all share columns (A, B, C, D) are 0
share_columns = ['share_adus_A', 'share_adus_B', 'share_adus_C', 'share_adus_D']
city_adus_pivot = city_adus_pivot.loc[city_adus_pivot[share_columns].any(axis=1)]
city_adus_pivot

In [ ]:
# Convert from wide to long format for Tableau
redlined_city_adus_long = city_adus_pivot.melt(
    id_vars=["city"],  # Keep city name
    value_vars=["share_adus_A", "share_adus_B", "share_adus_C", "share_adus_D"],  # Pivot these columns
    var_name="Redlining_Grade",  # New column for A, B, C, D, NA categories
    value_name="ADU_Share"  # New column for ADU share values
)

# Rename redlining grade values for clarity
redlined_city_adus_long['Redlining_Grade'] = redlined_city_adus_long['Redlining_Grade'].str.replace("share_adus_", "")
#redlined_city_adus_long['Redlining_Grade'] = redlined_city_adus_long['Redlining_Grade'].replace({"nan": "NA"})  # Rename NaN column

# Save the cleaned dataset for Tableau
redlined_city_adus_long
#redlined_city_adus_long.to_csv('city_adus_pivot_gradesonly_normalized.csv', index=False)

# Redlining by HOLC Area_ID

In [ ]:
import pandas as pd
holc_redlining = pd.read_csv("/home/jovyan/bac/redlined_maps_by_holc.csv")
holc_redlining.head()

In [ ]:
# Filter out rows where 'grade' is blank or NaN
holc_redlining = holc_redlining[holc_redlining['grade'].notna() & (holc_redlining['grade'] != '')]
#holc_redlining.to_csv('holc_redlining_filtered.csv')

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point

# Ensure Latitude and Longitude are numeric
holc_redlining['Latitude'] = pd.to_numeric(holc_redlining['Latitude'], errors='coerce')
holc_redlining['Longitude'] = pd.to_numeric(holc_redlining['Longitude'], errors='coerce')

# Group by area_id and create polygons
grouped = holc_redlining.groupby('area_id').agg({'Latitude': list, 'Longitude': list}).reset_index()

# Create polygon geometries
def create_polygon(latitudes, longitudes):
    points = [Point(lon, lat) for lon, lat in zip(longitudes, latitudes)]
    return Polygon(points) if len(points) > 2 else None

grouped['geometry'] = grouped.apply(lambda row: create_polygon(row['Latitude'], row['Longitude']), axis=1)
grouped = grouped.dropna()

# Convert to a GeoDataFrame
gdf = gpd.GeoDataFrame(grouped, geometry='geometry', crs="EPSG:4326")

# Plot the polygons
fig, ax = plt.subplots(figsize=(10, 8))
gdf.plot(ax=ax, color="red", alpha=0.5, edgecolor="black")
plt.title("Redlining Areas by area_id")
plt.show()


# Shares of each housing type by County

In [ ]:
# Count the occurrences of each housing type per county
county_housing_counts = hcd_2023_shares.groupby(['County', 'UNIT_CAT']).size()

# Get the total number of housing units per county
total_county_housing = hcd_2023_shares['County'].value_counts()

# Reindex total_county_housing to align with county_housing_counts
shares = county_housing_counts / total_county_housing.reindex(county_housing_counts.index.get_level_values('County')).values

# Convert to DataFrame
shares_df = shares.reset_index(name='Share')

shares_df.head()

# Permitted Units by Type in the Bay Area, 2023 (state of CA and by county)

In [ ]:
#use this dataset
hcd_2023_ALL

# Old Data

In [42]:
# HCD Table A2 Data from 2018-2020

hcd_old = pd.read_csv('/home/jovyan/bac/2018_20_hcd_data.csv')
hcd_old

,Longitude,Latitude,County Code,Tract Code,geoid,Share White,Share People of Color,Share Black,Share Asian,Share Hispanic/Latinx,...,NOTES,Activity,TOT_APPROVED_UNITS,TOT_DISAPPROVED_UNITS,APP_SUBMITTED_SB35_DESC,Activity Year,Income Category,Units,Income Sort Order,Activity Sort Order
0,-117.435695,34.580766,71.0,9116.0,6.071009e+09,13.4%,86.6%,25.0%,2.5%,57.8%,...,NaN,Permitted,NaN,NaN,NaN,2019,Above Moderate,1,4,3
1,-122.233975,37.756017,1.0,428200.0,6.001428e+09,51.8%,48.2%,6.3%,21.1%,12.8%,...,NaN,Permitted,NaN,NaN,NaN,2020,Above Moderate,1,4,3
2,-117.435483,34.579982,71.0,9116.0,6.071009e+09,13.4%,86.6%,25.0%,2.5%,57.8%,...,NaN,Permitted,NaN,NaN,NaN,2019,Above Moderate,1,4,3
3,-122.228540,37.755015,1.0,428100.0,6.001428e+09,52.8%,47.2%,6.5%,21.7%,11.6%,...,NaN,Permitted,NaN,NaN,NaN,2020,Above Moderate,1,4,3
4,-117.435685,34.579925,71.0,9116.0,6.071009e+09,13.4%,86.6%,25.0%,2.5%,57.8%,...,NaN,Permitted,NaN,NaN,NaN,2019,Above Moderate,1,4,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171062,-116.426843,34.147934,71.0,10411.0,6.071010e+09,70.6%,29.4%,2.9%,1.4%,18.5%,...,NaN,Permitted,NaN,NaN,NaN,2018,Above Moderate,1,4,3
171063,-116.433037,34.148097,71.0,10411.0,6.071010e+09,70.6%,29.4%,2.9%,1.4%,18.5%,...,NaN,Permitted,NaN,NaN,NaN,2018,Above Moderate,1,4,3
171064,-116.410208,34.136320,71.0,10411.0,6.071010e+09,70.6%,29.4%,2.9%,1.4%,18.5%,...,NaN,Permitted,NaN,NaN,NaN,2018,Above Moderate,1,4,3
171065,-116.435537,34.154861,71.0,10411.0,6.071010e+09,70.6%,29.4%,2.9%,1.4%,18.5%,...,NaN,Permitted,NaN,NaN,NaN,2018,Above Moderate,1,4,3


# Share and Number of ADUs in California, 2018-2023

In [43]:
hcd_data.head()

,JURIS_NAME,CNTY_NAME,YEAR,PRIOR_APN,APN,STREET_ADDRESS,PROJECT_NAME,JURS_TRACKING_ID,UNIT_CAT,TENURE,...,DEM_DES_UNITS,DEM_OR_DES_UNITS,DEM_DES_UNITS_OWN_RENT,DENSITY_BONUS_TOTAL,DENSITY_BONUS_NUMBER_OTHER_INCENTIVES,DENSITY_BONUS_INCENTIVES,DENSITY_BONUS_RECEIVE_REDUCTION,NOTES,Region,Type
0,ADELANTO,San Bernardino,2018,NaN,312846105,15824 BELLFLOWER ST,JOSE & ODULIA LOPEZ,NaN,SFD,Owner,...,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,Inland Deserts Region,Single-Family Detached Unit
1,ADELANTO,San Bernardino,2018,NaN,312878124,11939 GAINES CT,"FRONTIER HOMES II, LLC",NaN,SFD,Owner,...,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,Inland Deserts Region,Single-Family Detached Unit
2,ADELANTO,San Bernardino,2018,NaN,312878125,11931 GAINES CT,"FRONTIER HOMES II, LLC",NaN,SFD,Owner,...,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,Inland Deserts Region,Single-Family Detached Unit
3,ADELANTO,San Bernardino,2018,NaN,312878126,11925 GAINES CT,"FRONTIER HOMES II, LLC",NaN,SFD,Owner,...,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,Inland Deserts Region,Single-Family Detached Unit
4,ADELANTO,San Bernardino,2018,NaN,312878127,11913 GAINES CT,"FRONTIER HOMES II, LLC",NaN,SFD,Owner,...,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,Inland Deserts Region,Single-Family Detached Unit


In [44]:
# Filter observations with UNIT_CAT = 'ADU'
adu_data = hcd_data[hcd_data['UNIT_CAT'] == 'ADU']

# Group by year and count the number of ADU observations
adu_counts = adu_data.groupby('YEAR').size().reset_index(name='ADU_Count')

# Count the total observations by year
total_counts = hcd_data.groupby('YEAR').size().reset_index(name='Total_Count')

# Merge the two dataframes
summary = pd.merge(adu_counts, total_counts, on='YEAR')

# Calculate the share of ADU observations
summary['ADU_Share'] = summary['ADU_Count'] / summary['Total_Count']
summary.head()
#summary.to_csv('table_a2_share.csv', index=False)

,YEAR,ADU_Count,Total_Count,ADU_Share
0,2018,12786,84004,0.152207
1,2019,19259,97838,0.196846
2,2020,22586,97805,0.230929
3,2021,31934,119506,0.267217
4,2022,41346,124027,0.333363
